<a href="https://colab.research.google.com/github/kuzminadya/FHP-infovis-py/blob/master/Data_Dossier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Font File Transformations

## An idea

My idea is to analyse and visualise the transformation of font file changing over time. Creating a font, especially a large font family may take several years of work or it might be a result of collaboration of several designers and type engineers. Even after the first version is released, the contribution to the project and updating the font may last for several years. In my project, I would like to investigate what kind of changes had happened with a font file after it was published on GitHub.


### The first use case — analysis of a existed github repo

As an examle I use IBM Plex® typeface repo on GitHub:\
https://github.com/IBM/plex 

First of all I'm going to import our favorite libraries.


In [124]:
import pandas as pd
import altair as alt

The first step is getting the dataset in JSON format from Github (https://developer.github.com/v3/repos/#get-a-repository
). Here I have a description of repo itself.

In [125]:
import requests 

url = ('https://api.github.com/repos/IBM/plex')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_repo_json = response.json()

# dataset_repo_json

Request returned 200 : 'OK'


And then I get the list of commits (https://developer.github.com/v3/repos/commits/) in JSON format:

In [126]:
url = ('https://api.github.com/repos/IBM/plex/commits')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_commits_json = response.json()

# dataset_commits_json

Request returned 200 : 'OK'


The next part is converting JSON into a dataframe. I will convert only the second file.

With a help of: https://stackoverflow.com/questions/41168558/python-how-to-convert-json-file-to-dataframe

In [127]:
from pandas.io.json import json_normalize
df_commits = pd.DataFrame.from_dict(json_normalize(dataset_commits_json), orient='columns')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [128]:
df_commits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 59 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   sha                            30 non-null     object 
 1   node_id                        30 non-null     object 
 2   url                            30 non-null     object 
 3   html_url                       30 non-null     object 
 4   comments_url                   30 non-null     object 
 5   parents                        30 non-null     object 
 6   commit.author.name             30 non-null     object 
 7   commit.author.email            30 non-null     object 
 8   commit.author.date             30 non-null     object 
 9   commit.committer.name          30 non-null     object 
 10  commit.committer.email         30 non-null     object 
 11  commit.committer.date          30 non-null     object 
 12  commit.message                 30 non-null     objec

Some operations with df, like decreasing the amount of columns, haven't succeed with json file. To avoid it I transform the file in csv format and download it (to do it please uncomment the code below).

In [129]:
from google.colab import files

df_commits.to_csv('df_commits_table.csv')
# files.download('df_commits_table.csv')

In [130]:
file_commits = open("df_commits_table.csv")

file_commits_csv = pd.read_csv(file_commits, sep=',', keep_default_na=False)
file_commits_csv.head()

,Unnamed: 0,sha,node_id,url,html_url,comments_url,parents,commit.author.name,commit.author.email,commit.author.date,commit.committer.name,commit.committer.email,commit.committer.date,commit.message,commit.tree.sha,commit.tree.url,commit.url,commit.comment_count,commit.verification.verified,commit.verification.reason,commit.verification.signature,commit.verification.payload,author.login,author.id,author.node_id,author.avatar_url,author.gravatar_id,author.url,author.html_url,author.followers_url,author.following_url,author.gists_url,author.starred_url,author.subscriptions_url,author.organizations_url,author.repos_url,author.events_url,author.received_events_url,author.type,author.site_admin,committer.login,committer.id,committer.node_id,committer.avatar_url,committer.gravatar_id,committer.url,committer.html_url,committer.followers_url,committer.following_url,committer.gists_url,committer.starred_url,committer.subscriptions_url,committer.organizations_url,committer.repos_url,committer.events_url,committer.received_events_url,committer.type,committer.site_admin,author,committer
0,0,3fbdb5dd75d74379b7fba47dd889a1eb70771d73,MDY6Q29tbWl0MTA1NzAyNjMxOjNmYmRiNWRkNzVkNzQzNz...,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/3fbdb5dd75d...,https://api.github.com/repos/IBM/plex/commits/...,[{'sha': '968081d61b6abe18cd0dc14a42704afdf483...,Vince Picone,vpicone@gmail.com,2020-06-30T23:58:27Z,GitHub,noreply@github.com,2020-06-30T23:58:27Z,Merge pull request #323 from dtinth/patch-1\n\...,099315db66b26edb264f5b68047ac8ebff714a25,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,True,valid,-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQB...,tree 099315db66b26edb264f5b68047ac8ebff714a25\...,vpicone,4078018.0,MDQ6VXNlcjQwNzgwMTg=,https://avatars1.githubusercontent.com/u/40780...,,https://api.github.com/users/vpicone,https://github.com/vpicone,https://api.github.com/users/vpicone/followers,https://api.github.com/users/vpicone/following...,https://api.github.com/users/vpicone/gists{/gi...,https://api.github.com/users/vpicone/starred{/...,https://api.github.com/users/vpicone/subscript...,https://api.github.com/users/vpicone/orgs,https://api.github.com/users/vpicone/repos,https://api.github.com/users/vpicone/events{/p...,https://api.github.com/users/vpicone/received_...,User,False,web-flow,19864447.0,MDQ6VXNlcjE5ODY0NDQ3,https://avatars3.githubusercontent.com/u/19864...,,https://api.github.com/users/web-flow,https://github.com/web-flow,https://api.github.com/users/web-flow/followers,https://api.github.com/users/web-flow/followin...,https://api.github.com/users/web-flow/gists{/g...,https://api.github.com/users/web-flow/starred{...,https://api.github.com/users/web-flow/subscrip...,https://api.github.com/users/web-flow/orgs,https://api.github.com/users/web-flow/repos,https://api.github.com/users/web-flow/events{/...,https://api.github.com/users/web-flow/received...,User,False,,
1,1,f90d92225eda9e0056d13446cc979a83e11bd0a1,MDY6Q29tbWl0MTA1NzAyNjMxOmY5MGQ5MjIyNWVkYTllMD...,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/f90d92225ed...,https://api.github.com/repos/IBM/plex/commits/...,[{'sha': '968081d61b6abe18cd0dc14a42704afdf483...,Thai Pangsakulyanont,dtinth@spacet.me,2020-06-21T12:56:02Z,GitHub,noreply@github.com,2020-06-21T12:56:02Z,"Add ""main"" field to ""package.json""\n\nThis all...",099315db66b26edb264f5b68047ac8ebff714a25,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,True,valid,-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQB...,tree 099315db66b26edb264f5b68047ac8ebff714a25\...,dtinth,193136.0,MDQ6VXNlcjE5MzEzNg==,https://avatars3.githubusercontent.com/u/19313...,,https://api.github.com/users/dtinth,https://github.com/dtinth,https://api.github.com/users/dtinth/followers,https://api.github.com/users/dtinth/following{...,https://api.github.com/users/dtinth/gists{/gis...,h

In [131]:
file_commits_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 60 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Unnamed: 0                     30 non-null     int64 
 1   sha                            30 non-null     object
 2   node_id                        30 non-null     object
 3   url                            30 non-null     object
 4   html_url                       30 non-null     object
 5   comments_url                   30 non-null     object
 6   parents                        30 non-null     object
 7   commit.author.name             30 non-null     object
 8   commit.author.email            30 non-null     object
 9   commit.author.date             30 non-null     object
 10  commit.committer.name          30 non-null     object
 11  commit.committer.email         30 non-null     object
 12  commit.committer.date          30 non-null     object
 13  commit.

Converting a column with time-based data

In [132]:
file_commits_csv['commit.author.date'] = pd.to_datetime(file_commits_csv['commit.author.date'], format='%Y-%m-%d')

Reducing the amount of columns:



In [133]:
df_commits_reduced = file_commits_csv[["commit.author.name", "commit.author.date", "commit.message", "node_id"]]
df_commits_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   commit.author.name  30 non-null     object             
 1   commit.author.date  30 non-null     datetime64[ns, UTC]
 2   commit.message      30 non-null     object             
 3   node_id             30 non-null     object             
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 1.1+ KB


Renaming the columns. It was made in order to avoid periods in the column names. Thanks for Marian's help. Later I found an explanation https://altair-viz.github.io/user_guide/troubleshooting.html#encodings-with-special-characters

In [134]:
df_commits_reduced.columns = ["commit-author-name", "commit-author-date", "commit-message", "node_id"]

Adding a column with an int value in order to count the amount of items. Prabably a cheap trick


In [135]:
df_commits_reduced['value'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [136]:
df_commits_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   commit-author-name  30 non-null     object             
 1   commit-author-date  30 non-null     datetime64[ns, UTC]
 2   commit-message      30 non-null     object             
 3   node_id             30 non-null     object             
 4   value               30 non-null     int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 1.3+ KB


Here I made an overview to figure out how diverse is the list of contributors:

In [137]:
df_repo_groupby_name = df_commits_reduced.groupby('commit-author-name').count()
df_repo_groupby_name

,commit-author-date,commit-message,node_id,value
commit-author-name,,,,
LMapes,1,1,1,1
Mike Abbink,1,1,1,1
Thai Pangsakulyanont,1,1,1,1
Vince Picone,27,27,27,27


## Visualisations



Let's visualise the dataset. Since I'm presenting a dataset that changes over time I've choosen streamgraph to show the repo overview.

In [138]:
alt.Chart(df_commits_reduced).mark_area(interpolate='monotone',
    fillOpacity=0.8,
    stroke='white',
    strokeWidth=0.2).encode(
    alt.X('monthdate(commit-author-date):T',
        axis=alt.Axis(format='%d.%m.%Y', domain=False, tickSize=0, title='Date')
    ),
    alt.Y('sum(value):Q', stack='center', axis=None, title='Commits'),
    alt.Color('commit-author-name:N', title='Author Name',
        scale=alt.Scale(scheme='category20b')
    ),
    tooltip = ['commit-author-name:N'],
).properties(
    width=800,
    height=400
).interactive()

alt.Chart(...)

To find out how many commits were made by each particular author it is possible to visualise a part of the dataset with a simple bar chart. At the same it is interesting to figure out when exactly during the day each commit was posted. For this purpose I used simple scatterplot. With the ineractive tooltip option it is easy to dive into details and see the author name together with the commit message. 

In [139]:
base = alt.Chart(df_commits_reduced).properties(width=400, height=300)

scat = base.mark_circle(size=80, opacity=0.5).encode(
    alt.Y('hoursminutes(commit-author-date):T', title='Hour'),
    alt.X('monthdate(commit-author-date):T', title='Date'),
    alt.Color('commit-author-name:N', title='Name of the author'),
    tooltip = ['commit-author-name:N', 'commit-message:N']
).interactive()

bar = base.mark_bar().encode(
    alt.X('commit-author-name:O', title='Name of the author', sort='-x', axis=alt.Axis(labelAngle=300)),
    alt.Y('count_val:Q', title='Commits'),
    color='commit-author-name:N',
).transform_aggregate(
    count_val='count(value)',
    groupby=['commit-author-name']
).interactive()

bar | scat

alt.HConcatChart(...)

A next step can be the analysis of what commit messages contain. For this purpose I'm importing Natural Language Toolkit.\
Based on https://colab.research.google.com/drive/1y5j8DoczO57bUzUBbqqKd5lU2j_GlCuG?usp=sharing

In [140]:
import nltk

In [141]:
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In order to extract information from a particular column I should join all cells into one string.\
based on https://colab.research.google.com/drive/1y5j8DoczO57bUzUBbqqKd5lU2j_GlCuG?usp=sharing

In [142]:
df_commits_string = df_commits_reduced["commit-message"].tolist() 
df_commits_string = "".join(df_commits_string)
df_commits_string

'Merge pull request #323 from dtinth/patch-1\n\nAdd "main" field to "package.json"Add "main" field to "package.json"\n\nThis allows the package’s CSS to be consumed directly, following how the typefaces project does it: https://github.com/KyleAMathews/typefaces5.0.0Merge pull request #315 from vpicone/cross-language-updatefix: tweak zip script to handle korean and variablefix: temporarily exclude korean from zipfeat: add koreanfix: change internal family name for condensed, add glyphsfix(Serif): vertical metrics and glyph additionsfix(Sans): new glyphs, fixed vertical metricsfeat: mono updateUpdate README.md (#310)\n\n* Update README.md\n\nThis change includes new non-Latin fonts and updates content based on the IBM Style guide.\n\n* Update README.md\n\nHi Vince, this is ready to merge now.\n\n* Update README.md\n\nChange to one sentence per Mike Abbink.4.0.2fix: update using new hebrew fixes (#289)\n\nfix: update using new hebrew fixesfix: update using new hebrew fixesUpdate README.md

The next step is tokenisation:

In [143]:
tokens = nltk.word_tokenize(df_commits_string)
tokens[0:10]

['Merge',
 'pull',
 'request',
 '#',
 '323',
 'from',
 'dtinth/patch-1',
 'Add',
 '``',
 'main']

Then counting words without stopwords:

In [144]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as stop

stopwords = stop.words("english")

print(stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

Counting an amount of words in each group:


In [145]:
words = [word for word in tokens if word not in stopwords]

bow = {}

for word in words:
  bow[word] = words.count(word)

words_frequency = sorted(bow.items(), key=lambda x: x[1], reverse=True)

print(words_frequency[0:100])

[(':', 19), ('update', 8), ('#', 7), ('(', 7), (')', 7), ('new', 5), ('README.md', 5), ('``', 4), ("''", 4), (',', 4), ('add', 4), ('family', 4), ('name', 4), ('Update', 4), ('thai', 4), ('*', 3), ('using', 3), ('hebrew', 3), ('arabic', 3), ('include', 3), ('pull', 2), ('request', 2), ('Add', 2), ('main', 2), ('field', 2), ('package.json', 2), ('This', 2), ('korean', 2), ('change', 2), ('vertical', 2), ('Sans', 2), ('.', 2), ('README.mdUpdate', 2), ('Merge', 1), ('323', 1), ('dtinth/patch-1', 1), ('allows', 1), ('package', 1), ('’', 1), ('CSS', 1), ('consumed', 1), ('directly', 1), ('following', 1), ('typefaces', 1), ('project', 1), ('https', 1), ('//github.com/KyleAMathews/typefaces5.0.0Merge', 1), ('315', 1), ('vpicone/cross-language-updatefix', 1), ('tweak', 1), ('zip', 1), ('script', 1), ('handle', 1), ('variablefix', 1), ('temporarily', 1), ('exclude', 1), ('zipfeat', 1), ('koreanfix', 1), ('internal', 1), ('condensed', 1), ('glyphsfix', 1), ('Serif', 1), ('metrics', 1), ('glyph',

And the visualisation itself. A simple sorted bar chart which shows the frequency of appearing of each word in commits comments.

In [146]:
df = pd.DataFrame(words_frequency, columns=['word', 'count'])
 
alt.Chart(df).mark_bar().encode(
  x = 'count:Q',
  y=alt.Y('word:N', sort='-x')
)

alt.Chart(...)

### Conclution

In general I would like to thank you Marian for this brilliant course. As to myself I find the biggest challange in collecting and preparing your own dataset. During my work I faced the problem of incompartible data type too often. I can only imagine the reason is in the converting json file into csv and then into pandas dataframe. Still this course gave me important knowledge and motivation to dive deeper in the topic.\